In [1]:
%%capture
!pip install rioxarray  xarray

In [2]:
import os
import yaml
from datetime import datetime,timedelta
from yaml.loader import SafeLoader
import rioxarray as rx
import pandas as pd
import boto3
import xarray as xr
import glob
import shutil
import os
from dask.diagnostics import ProgressBar
from datetime import datetime
import numpy as np
from dask import dataframe as dd

In [3]:
twoMonthsDate = (datetime.now()-timedelta(days=60))
year = twoMonthsDate.strftime('%Y')
month = twoMonthsDate.strftime('%m')
ymlPath = 'conf/chirps.yml'

In [4]:
def readYamlConf(path: str):
    with open(path,'r') as conf:
        confArgs = yaml.load(conf, Loader= SafeLoader)
        #print(confArgs)
    return confArgs

def downloadDataFromS3FilteredByYear(s3Path: str
                                     , downloadPath: str
                                     , year:str 
                                     , month : str
                                    ):
    
    s3Command = f'aws s3 cp {s3Path} {downloadPath} --recursive --exclude "*" --include "chirps-v2.0.{year}.{month}.*"'
    
    return os.system(s3Command)

def uncompressGunzip (path:str):
    command = f'cd {path}| gunzip {path}*'
    os.system(command)
    

def VtransformCHIRTSToNc(inputPath, file, outputPath):
    tiffFile = inputPath+file
    allFiles = xr.open_dataset(tiffFile)
    dateContent = file.split('.')
    date = datetime(int(dateContent[-4]),int(dateContent[-3]),int(dateContent[-2]))
    filename = date.strftime('%Y%m%d')
    time_da = xr.DataArray({'time': date})
    allFiles = allFiles.assign_coords(time = date)
    allFiles = allFiles.expand_dims(dim="time")
    allFiles = allFiles.rename_vars(name_dict = {'band_data':'precipitation'})
    allFiles = allFiles.rename(name_dict   = {'x':'lon','y':'lat'})
    allFiles = allFiles.drop('band')
    allFiles = allFiles.drop('spatial_ref')
    allFiles.to_netcdf(f'{outputPath}{filename}.nc')
    

In [5]:
confArgs = readYamlConf(ymlPath)
chirpsArgs = confArgs['chirpsArguments']
del confArgs

In [6]:
logCapture = downloadDataFromS3FilteredByYear(chirpsArgs['awsChirpsPath']
                                 , chirpsArgs['pathforTGZ']
                                 ,'2023'
                                 ,'07'
                                )


download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.02.tif.gz to tgz/chirps-v2.0.2023.07.02.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.08.tif.gz to tgz/chirps-v2.0.2023.07.08.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.06.tif.gz to tgz/chirps-v2.0.2023.07.06.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.03.tif.gz to tgz/chirps-v2.0.2023.07.03.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.09.tif.gz to tgz/chirps-v2.0.2023.07.09.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/chirps-v2.0.2023.07.07.tif.gz to tgz/chirps-v2.0.2023.07.07.tif.gz
download: s3://climate-action-datalake/zone=landing/source=CHIRP

In [7]:
uncompressGunzip(chirpsArgs['pathforTGZ'])

In [8]:
tiffToTransform = os.listdir(chirpsArgs['pathforTGZ'])

In [9]:
tiffToTransformDf = pd.DataFrame()
tiffToTransformDf['files'] = tiffToTransform


In [10]:
tiffToTransformDf

,files
0,chirps-v2.0.2023.07.09.tif
1,chirps-v2.0.2023.07.12.tif
2,chirps-v2.0.2023.07.15.tif
3,chirps-v2.0.2023.07.10.tif
4,chirps-v2.0.2023.07.26.tif
5,chirps-v2.0.2023.07.23.tif
6,chirps-v2.0.2023.07.13.tif
7,chirps-v2.0.2023.07.31.tif
8,chirps-v2.0.2023.07.17.tif
9,chirps-v2.0.2023.07.25.tif


In [11]:
tiffToTransformDf.files.apply(lambda f: VtransformCHIRTSToNc(chirpsArgs['pathforTGZ']
                                                            , f
                                                            , chirpsArgs['pathforNCDF'])
                            )

/tmp/ipykernel_14607/2728258440.py:33: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  allFiles = allFiles.drop('band')
/tmp/ipykernel_14607/2728258440.py:34: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  allFiles = allFiles.drop('spatial_ref')
/tmp/ipykernel_14607/2728258440.py:33: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  allFiles = allFiles.drop('band')
/tmp/ipykernel_14607/2728258440.py:34: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  allFiles = allFiles.drop('spatial_ref')
/tmp/ipykernel_14607/2728258440.py:33: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  allFiles = allFiles.drop('band')
/tmp/ipykernel_14607/2728258440.py:34: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  allFiles = allFiles.drop('spatial_ref')
/tmp/ipykernel_14607/2728258440.py:33: Depr

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
Name: files, dtype: object

In [12]:
chirpsArgs

{'pathforTGZ': '/home/ec2-user/SageMaker/datalake/notebooks/chirps/tgz/',
 'pathforNCDF': '/home/ec2-user/SageMaker/datalake/notebooks/chirps/nc/',
 'awsChirpsPath': 's3://climate-action-datalake/zone=landing/source=CHIRPS/variable=precipitation/',
 'awsChirpsTemporalPath': 's3://climate-action-datalake/zone=temporal/source=chirps/variable=precipitation/'}